In [250]:
# Importación de librerías
import os
import pickle
import numpy as np
import glob
import keras.backend as K 
import keras
import keras.utils
from keras import utils as np_utils
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from music21 import *
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import plot_model
from keras.layers import LSTM, Input, Dropout, Dense, Activation, Embedding, Concatenate, Reshape
from keras.layers import Flatten, RepeatVector, Permute, TimeDistributed
from keras.layers import Multiply, Lambda, Softmax
from keras.models import Model
from keras.optimizers import RMSprop
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import load_model, Model
from keras.layers import Attention
from keras.layers import Bidirectional

In [251]:
# Paramentros para la creación de los direcctorios donde se almacenará las salidas y los pesos del entrenamiento
seccion = 'composicion'
id_ejecucion = '0001'
nombre_musical = 'guitarra'

directorio_ejecucion = './ejecucion/{}/'.format(seccion)
directorio_ejecucion += '_'.join([id_ejecucion, nombre_musical])

directorio_almacenamiento = os.path.join(directorio_ejecucion, 'almacenamiento')
directorio_partituras = os.path.join('datos', nombre_musical)

# Si no están creados los directorios los creamos
if not os.path.exists(directorio_ejecucion):
    os.mkdir(directorio_ejecucion)
    os.mkdir(os.path.join(directorio_ejecucion, 'almacenamiento'))
    os.mkdir(os.path.join(directorio_ejecucion, 'output'))
    os.mkdir(os.path.join(directorio_ejecucion, 'pesos'))
    os.mkdir(os.path.join(directorio_ejecucion, 'viz'))

In [252]:
############################################
# Modo de ejecución
#############################################
modo_ejecucion = 'creacion'  
############################################

#############################################
# Parametros para la generación de secuencias
#############################################
intervalos = range(1) # Se utiliza para data aumentation. 
                      # Se generá la misma paritura pero incrementada cada vez en 1 semitono
longitud_secuencia = 16 
#############################################

#############################################
# Parámetros del modelo
#############################################
# model params
tamanio_embedding = 128 # Número de embedding a utilizar
neuronas_LSTM = 128 # Numero de neuronas por capa LSTM
usar_attention = False # Si se va a utilizar mecanismo de Attention
#############################################

In [253]:
# Recupera la lista de partituras MIDI de un directorio
def recuperar_lista_partituras(directorio_partituras):
    lista_partituras = glob.glob(os.path.join(directorio_partituras, "*.mid"))
    parseador = converter
    return lista_partituras, parseador

In [254]:
if modo_ejecucion == 'creacion':
    
    # Se recupera la lista de partituras del directorio
    lista_partituras_musica, parseador = recuperar_lista_partituras(directorio_partituras)
    print(len(lista_partituras_musica), 'Total de ficheros encontrados')

    notas = []
    duraciones = []

    # Para cada partitura encontrada se realiza el parseo consistente en obtener nota y duración
    for i, fichero in enumerate(lista_partituras_musica):
        print(i+1, "Parseando %s" % fichero)
        partitura = parseador.parse(fichero).chordify()
     
        # Para cada intervalo se genera una nueva partitura igual que la anterior pero con los notas
        # traspuesta un semitono hacía arriaba.
        # Esto permite realiza data aumentation si el valor de intervalo es mayor que 1
        for intervalo in intervalos:
            # Se realiza la trasposición de las notas y se sube tantos semitonos como indique intervalo.
            # Si el intervalo es 0 no se transpone la partitura.
            evento = partitura.transpose(intervalo)

            # Se añaden tantos START iniciales con longitud tenga la secuencia
            notas.extend(['START'] * longitud_secuencia)

            # Se añaden tantos 0 como longitud tenga la secuencia ya que el evento START tiene duración 0
            duraciones.extend([0]* longitud_secuencia)

            # Por cada evento que se encuentre se analiza si es una nota simple, un silencio o un conjunto de notas (acorde)
            for elemento in evento.flat:
                # Si es una nota
                if isinstance(elemento, note.Note):
                    # Si es un silencio
                    notas.append(str(elemento.nameWithOctave))
                    duraciones.append(elemento.duration.quarterLength)
                    #print('Nota:', str(elemento.nameWithOctave), elemento.duration.quarterLength)
                       
                # Si es un acorde
                if isinstance(elemento, chord.Chord):
                    notas.append('.'.join(n.nameWithOctave for n in elemento.pitches))
                    duraciones.append(elemento.duration.quarterLength)
                    #print('Acorde:', '.'.join(n.nameWithOctave for n in elemento.pitches), elemento.duration.quarterLength)
                
                # Si es un silencio
                if isinstance(elemento, note.Rest):
                    notas.append(str(elemento.name))
                    duraciones.append(elemento.duration.quarterLength)
                    #print('Silencio:', str(elemento.name), elemento.duration.quarterLength)

    # Se guarda la información de notas y sus duraciones en el directorio de almacenamiento
    with open(os.path.join(directorio_almacenamiento, 'notas'), 'wb') as fich:
        pickle.dump(notas, fich) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open(os.path.join(directorio_almacenamiento, 'duraciones'), 'wb') as fich:
        pickle.dump(duraciones, fich) 
else:
    # Si el modo de ejecución no es 'creacion' entonces se recuperan las notas y sus duraciones 
    # del directorio de almacenamiento
    with open(os.path.join(directorio_almacenamiento, 'notas'), 'rb') as fich:
        notas = pickle.load(fich) 
    with open(os.path.join(directorio_almacenamiento, 'duraciones'), 'rb') as fich:
        duraciones = pickle.load(fich)

40 Total de ficheros encontrados
1 Parseando datos\guitarra\solea_input_01_mono.mid
2 Parseando datos\guitarra\solea_input_02_mono.mid
3 Parseando datos\guitarra\solea_input_03_mono.mid
4 Parseando datos\guitarra\solea_input_04_mono.mid
5 Parseando datos\guitarra\solea_input_05_mono.mid
6 Parseando datos\guitarra\solea_input_06_mono.mid
7 Parseando datos\guitarra\solea_input_07_mono.mid
8 Parseando datos\guitarra\solea_input_08_mono.mid
9 Parseando datos\guitarra\solea_input_09_mono.mid
10 Parseando datos\guitarra\solea_input_10_mono.mid
11 Parseando datos\guitarra\solea_input_11_mono.mid
12 Parseando datos\guitarra\solea_input_12_mono.mid
13 Parseando datos\guitarra\solea_input_13_mono.mid
14 Parseando datos\guitarra\solea_input_14_mono.mid
15 Parseando datos\guitarra\solea_input_15_mono.mid
16 Parseando datos\guitarra\solea_input_16_mono.mid
17 Parseando datos\guitarra\solea_input_18_mono.mid
18 Parseando datos\guitarra\solea_input_19_mono.mid
19 Parseando datos\guitarra\solea_input_

In [255]:
# Lista que Contiene las notas
set(notas)

{'A2',
 'A3',
 'A4',
 'B-2',
 'B-3',
 'B-4',
 'B2',
 'B3',
 'B4',
 'C#3',
 'C#4',
 'C#5',
 'C3',
 'C4',
 'C5',
 'D3',
 'D4',
 'D5',
 'E-3',
 'E-4',
 'E2',
 'E2.B3',
 'E2.C4',
 'E2.D4',
 'E3',
 'E4',
 'E5',
 'F#2',
 'F#3',
 'F#4',
 'F2',
 'F3',
 'F4',
 'F5',
 'G#2',
 'G#3',
 'G#4',
 'G2',
 'G3',
 'G4',
 'START',
 'rest'}

In [256]:
# Lista que contiene las duraciones
set(duraciones)

{0,
 Fraction(1, 6),
 0.25,
 Fraction(1, 3),
 0.5,
 Fraction(2, 3),
 0.75,
 1.0,
 1.25,
 Fraction(4, 3),
 1.5,
 2.0,
 2.5,
 3.0,
 4.0}

In [257]:
# Se redondea el valor de la duración a 8 decimales y se elimina la definicion con fraccion
for i in range(len(duraciones)):
    duraciones[i]=round(float(duraciones[i]), 8)

print('duraciones_set', set(duraciones))

duraciones_set {0.0, 0.25, 0.33333333, 0.5, 1.0, 3.0, 4.0, 0.75, 2.0, 0.66666667, 1.33333333, 1.25, 1.5, 2.5, 0.16666667}


In [258]:
# Añade el compas flamenco de la solea como una nueva lista.
# Esta información se añadira a la entrada del entrenamiento para que la red pueda aprender el compas

compas_flamenco=[]
acumulator_compas=1
compas_solea=12

# Como el compas de la solea es de 12 tiempos cambiamos a 1 cuando llegamos al 13
for i in range(len(duraciones)):

    if (acumulator_compas >= compas_solea + 1):
        acumulator_compas= acumulator_compas - compas_solea
        
    compas_flamenco.append(int(acumulator_compas))
    print(notas[i],'--', duraciones[i],'--',acumulator_compas,'--', int(acumulator_compas))

    acumulator_compas = round(acumulator_compas + duraciones[i],2)
    
    # En los casos de tresillos, cinquillos, seisillos, etc ajustamos el compas por los decimales
    if (int(acumulator_compas + 0.01) == round(acumulator_compas,1)):
        acumulator_compas=int(acumulator_compas + 0.01)
    

START -- 0.0 -- 1 -- 1
START -- 0.0 -- 1 -- 1
START -- 0.0 -- 1 -- 1
START -- 0.0 -- 1 -- 1
START -- 0.0 -- 1 -- 1
START -- 0.0 -- 1 -- 1
START -- 0.0 -- 1 -- 1
START -- 0.0 -- 1 -- 1
START -- 0.0 -- 1 -- 1
START -- 0.0 -- 1 -- 1
START -- 0.0 -- 1 -- 1
START -- 0.0 -- 1 -- 1
START -- 0.0 -- 1 -- 1
START -- 0.0 -- 1 -- 1
START -- 0.0 -- 1 -- 1
START -- 0.0 -- 1 -- 1
A2 -- 0.25 -- 1 -- 1
C4 -- 0.25 -- 1.25 -- 1
E3 -- 0.25 -- 1.5 -- 1
A3 -- 0.25 -- 1.75 -- 1
C4 -- 0.25 -- 2 -- 2
E3 -- 0.25 -- 2.25 -- 2
D3 -- 0.25 -- 2.5 -- 2
C3 -- 0.25 -- 2.75 -- 2
D3 -- 0.25 -- 3 -- 3
C3 -- 0.25 -- 3.25 -- 3
B2 -- 0.25 -- 3.5 -- 3
A2 -- 0.25 -- 3.75 -- 3
G2 -- 0.25 -- 4 -- 4
B3 -- 0.25 -- 4.25 -- 4
B2 -- 0.25 -- 4.5 -- 4
D3 -- 0.25 -- 4.75 -- 4
G3 -- 0.25 -- 5 -- 5
E3 -- 0.25 -- 5.25 -- 5
D3 -- 0.25 -- 5.5 -- 5
C3 -- 0.25 -- 5.75 -- 5
D3 -- 0.25 -- 6 -- 6
B2 -- 0.25 -- 6.25 -- 6
A2 -- 0.25 -- 6.5 -- 6
G2 -- 0.25 -- 6.75 -- 6
F2 -- 0.25 -- 7 -- 7
A2 -- 0.25 -- 7.25 -- 7
D3 -- 0.25 -- 7.5 -- 7
E3 -- 0.25 -

C4 -- 0.25 -- 8.25 -- 8
F4 -- 0.25 -- 8.5 -- 8
A4 -- 0.25 -- 8.75 -- 8
G4 -- 0.25 -- 9 -- 9
F4 -- 0.25 -- 9.25 -- 9
E4 -- 0.25 -- 9.5 -- 9
D4 -- 0.25 -- 9.75 -- 9
E2 -- 0.5 -- 10 -- 10
E4 -- 0.5 -- 10.5 -- 10
E3 -- 0.5 -- 11 -- 11
G#3 -- 0.5 -- 11.5 -- 11
B3 -- 1.0 -- 12 -- 12
A2 -- 0.25 -- 1 -- 1
E5 -- 0.25 -- 1.25 -- 1
A4 -- 0.25 -- 1.5 -- 1
E4 -- 0.25 -- 1.75 -- 1
C4 -- 0.25 -- 2 -- 2
E4 -- 0.25 -- 2.25 -- 2
A4 -- 0.25 -- 2.5 -- 2
E5 -- 0.25 -- 2.75 -- 2
B4 -- 0.25 -- 3 -- 3
C5 -- 0.25 -- 3.25 -- 3
D5 -- 0.25 -- 3.5 -- 3
B4 -- 0.25 -- 3.75 -- 3
G3 -- 0.25 -- 4 -- 4
B4 -- 0.25 -- 4.25 -- 4
G4 -- 0.25 -- 4.5 -- 4
D4 -- 0.25 -- 4.75 -- 4
B3 -- 0.25 -- 5 -- 5
D4 -- 0.25 -- 5.25 -- 5
G4 -- 0.25 -- 5.5 -- 5
B4 -- 0.25 -- 5.75 -- 5
A4 -- 0.25 -- 6 -- 6
B4 -- 0.25 -- 6.25 -- 6
C5 -- 0.25 -- 6.5 -- 6
A4 -- 0.25 -- 6.75 -- 6
F3 -- 0.25 -- 7 -- 7
A4 -- 0.25 -- 7.25 -- 7
F4 -- 0.25 -- 7.5 -- 7
C4 -- 0.25 -- 7.75 -- 7
A3 -- 0.25 -- 8 -- 8
C4 -- 0.25 -- 8.25 -- 8
F4 -- 0.25 -- 8.5 -- 8
A4 -- 0.25

A2 -- 0.25 -- 4.75 -- 4
G2 -- 0.25 -- 5 -- 5
B3 -- 0.25 -- 5.25 -- 5
B2 -- 0.25 -- 5.5 -- 5
D3 -- 0.25 -- 5.75 -- 5
G3 -- 0.25 -- 6 -- 6
E3 -- 0.25 -- 6.25 -- 6
D3 -- 0.25 -- 6.5 -- 6
C3 -- 0.25 -- 6.75 -- 6
D3 -- 0.25 -- 7 -- 7
B2 -- 0.25 -- 7.25 -- 7
A2 -- 0.25 -- 7.5 -- 7
G2 -- 0.25 -- 7.75 -- 7
F2 -- 0.25 -- 8 -- 8
A2 -- 0.25 -- 8.25 -- 8
D3 -- 0.25 -- 8.5 -- 8
E3 -- 0.25 -- 8.75 -- 8
D3 -- 0.25 -- 9 -- 9
C3 -- 0.25 -- 9.25 -- 9
A2 -- 0.25 -- 9.5 -- 9
G#2 -- 0.25 -- 9.75 -- 9
A2 -- 0.25 -- 10 -- 10
F2 -- 0.25 -- 10.25 -- 10
G2 -- 0.25 -- 10.5 -- 10
F2 -- 0.25 -- 10.75 -- 10
E2 -- 0.5 -- 11 -- 11
E4 -- 0.5 -- 11.5 -- 11
E3 -- 0.5 -- 12 -- 12
G#3 -- 0.5 -- 12.5 -- 12
B3 -- 0.5 -- 1 -- 1
E4 -- 0.5 -- 1.5 -- 1
E3 -- 0.5 -- 2 -- 2
E4 -- 0.5 -- 2.5 -- 2
D3 -- 0.5 -- 3 -- 3
E4 -- 0.5 -- 3.5 -- 3
F3 -- 0.5 -- 4 -- 4
B3 -- 0.25 -- 4.5 -- 4
A3 -- 0.25 -- 4.75 -- 4
B3 -- 0.25 -- 5 -- 5
C4 -- 0.25 -- 5.25 -- 5
D4 -- 0.25 -- 5.5 -- 5
E4 -- 0.25 -- 5.75 -- 5
F4 -- 0.25 -- 6 -- 6
E4 -- 0.25 -- 6.

C4 -- 0.25 -- 4.25 -- 4
B3 -- 0.25 -- 4.5 -- 4
A3 -- 0.25 -- 4.75 -- 4
G#3 -- 0.25 -- 5 -- 5
A3 -- 0.25 -- 5.25 -- 5
G#3 -- 0.25 -- 5.5 -- 5
F3 -- 0.25 -- 5.75 -- 5
E3 -- 0.25 -- 6 -- 6
F3 -- 0.25 -- 6.25 -- 6
G#3 -- 0.25 -- 6.5 -- 6
A3 -- 0.25 -- 6.75 -- 6
B3 -- 0.5 -- 7 -- 7
D4 -- 0.5 -- 7.5 -- 7
D4 -- 0.25 -- 8 -- 8
C4 -- 0.25 -- 8.25 -- 8
B3 -- 0.25 -- 8.5 -- 8
A3 -- 0.25 -- 8.75 -- 8
G#3 -- 0.25 -- 9 -- 9
A3 -- 0.25 -- 9.25 -- 9
B3 -- 0.25 -- 9.5 -- 9
C4 -- 0.25 -- 9.75 -- 9
B3 -- 0.25 -- 10 -- 10
C4 -- 0.25 -- 10.25 -- 10
B3 -- 0.25 -- 10.5 -- 10
A3 -- 0.25 -- 10.75 -- 10
B3 -- 3.0 -- 11 -- 11
A2 -- 0.25 -- 2 -- 2
A3 -- 0.25 -- 2.25 -- 2
E3 -- 0.25 -- 2.5 -- 2
C3 -- 0.25 -- 2.75 -- 2
E3 -- 0.25 -- 3 -- 3
A3 -- 0.25 -- 3.25 -- 3
B3 -- 0.25 -- 3.5 -- 3
C4 -- 0.25 -- 3.75 -- 3
B3 -- 0.25 -- 4 -- 4
C4 -- 0.25 -- 4.25 -- 4
B3 -- 0.25 -- 4.5 -- 4
A3 -- 0.25 -- 4.75 -- 4
G2 -- 0.25 -- 5 -- 5
G3 -- 0.25 -- 5.25 -- 5
D3 -- 0.25 -- 5.5 -- 5
B2 -- 0.25 -- 5.75 -- 5
D3 -- 0.25 -- 6 -- 6
G3 -

A2 -- 0.33333333 -- 8.33 -- 8
B-2 -- 0.33333333 -- 8.66 -- 8
B2 -- 0.33333333 -- 9 -- 9
C3 -- 0.33333333 -- 9.33 -- 9
C#3 -- 0.33333333 -- 9.66 -- 9
C3 -- 0.33333333 -- 10 -- 10
B-2 -- 0.33333333 -- 10.33 -- 10
G#2 -- 0.33333333 -- 10.66 -- 10
G2 -- 0.33333333 -- 11 -- 11
D3 -- 0.33333333 -- 11.33 -- 11
F3 -- 0.33333333 -- 11.66 -- 11
G#3 -- 0.33333333 -- 12 -- 12
F3 -- 0.33333333 -- 12.33 -- 12
D3 -- 0.33333333 -- 12.66 -- 12
B2 -- 1.0 -- 1 -- 1
C3 -- 0.33333333 -- 2 -- 2
D3 -- 0.33333333 -- 2.33 -- 2
C3 -- 0.33333333 -- 2.66 -- 2
B2 -- 0.33333333 -- 3 -- 3
B-2 -- 0.33333333 -- 3.33 -- 3
A2 -- 0.33333333 -- 3.66 -- 3
G#2 -- 0.5 -- 4 -- 4
B3 -- 0.25 -- 4.5 -- 4
E-3 -- 0.25 -- 4.75 -- 4
G#3 -- 0.33333333 -- 5 -- 5
C4 -- 0.33333333 -- 5.33 -- 5
G#2 -- 0.33333333 -- 5.66 -- 5
G2 -- 0.33333333 -- 6 -- 6
G#2 -- 0.33333333 -- 6.33 -- 6
A2 -- 0.33333333 -- 6.66 -- 6
B-2 -- 0.33333333 -- 7 -- 7
B-3 -- 0.33333333 -- 7.33 -- 7
B-2 -- 0.33333333 -- 7.66 -- 7
B-2 -- 0.33333333 -- 8 -- 8
B2 -- 0.33

...

E3 -- 0.5 -- 6 -- 6
G#3 -- 0.5 -- 6.5 -- 6
B3 -- 0.5 -- 7 -- 7
E4 -- 0.5 -- 7.5 -- 7
E2 -- 0.33333333 -- 8 -- 8
G#2 -- 0.33333333 -- 8.33 -- 8
B2 -- 0.33333333 -- 8.66 -- 8
D3 -- 0.33333333 -- 9 -- 9
E3 -- 0.33333333 -- 9.33 -- 9
F3 -- 0.33333333 -- 9.66 -- 9
E3 -- 0.33333333 -- 10 -- 10
D3 -- 0.33333333 -- 10.33 -- 10
C3 -- 0.33333333 -- 10.66 -- 10
B2 -- 0.33333333 -- 11 -- 11
A2 -- 0.33333333 -- 11.33 -- 11
G#2 -- 0.33333333 -- 11.66 -- 11
A2 -- 0.5 -- 12 -- 12
A2 -- 0.5 -- 12.5 -- 12
A2 -- 0.5 -- 1 -- 1
A2 -- 0.5 -- 1.5 -- 1
G2 -- 0.5 -- 2 -- 2
F2 -- 0.5 -- 2.5 -- 2
G2 -- 0.33333333 -- 3 -- 3
F2 -- 0.33333333 -- 3.33 -- 3
E2 -- 0.33333333 -- 3.66 -- 3
F2 -- 1.0 -- 4 -- 4
E2 -- 0.5 -- 5 -- 5
F2 -- 0.5 -- 5.5 -- 5
F2 -- 0.5 -- 6 -- 6
G2 -- 0.5 -- 6.5 -- 6
A2 -- 1.0 -- 7 -- 7
E2 -- 0.5 -- 8 -- 8
E4 -- 0.5 -- 8.5 -- 8
E3 -- 0.5 -- 9 -- 9
G#3 -- 0.5 -- 9.5 -- 9
B3 -- 0.5 -- 10 -- 10
E4 -- 0.5 -- 10.5 -- 10
A2 -- 0.25 -- 11 -- 11
A3 -- 0.25 -- 11.25 -- 11
C4 -- 0.25 -- 11.5 -- 11
E4 -- 0

F2 -- 0.33333333 -- 5.33 -- 5
A2 -- 0.33333333 -- 5.66 -- 5
B2 -- 0.33333333 -- 6 -- 6
C3 -- 0.33333333 -- 6.33 -- 6
D3 -- 0.33333333 -- 6.66 -- 6
E3 -- 0.33333333 -- 7 -- 7
G2 -- 0.33333333 -- 7.33 -- 7
A2 -- 0.33333333 -- 7.66 -- 7
B2 -- 0.33333333 -- 8 -- 8
C3 -- 0.33333333 -- 8.33 -- 8
D3 -- 0.33333333 -- 8.66 -- 8
E3 -- 0.33333333 -- 9 -- 9
F3 -- 0.33333333 -- 9.33 -- 9
G3 -- 0.33333333 -- 9.66 -- 9
A3 -- 1.0 -- 10 -- 10
E3 -- 0.33333333 -- 11 -- 11
F3 -- 0.33333333 -- 11.33 -- 11
G3 -- 0.33333333 -- 11.66 -- 11
A3 -- 0.33333333 -- 12 -- 12
C4 -- 0.33333333 -- 12.33 -- 12
B3 -- 0.33333333 -- 12.66 -- 12
A3 -- 0.33333333 -- 1 -- 1
G3 -- 0.33333333 -- 1.33 -- 1
F3 -- 0.33333333 -- 1.66 -- 1
E2 -- 0.5 -- 2 -- 2
E4 -- 0.5 -- 2.5 -- 2
E3 -- 0.5 -- 3 -- 3
G#3 -- 0.5 -- 3.5 -- 3
E4 -- 1.0 -- 4 -- 4
G2 -- 0.33333333 -- 5 -- 5
C3 -- 0.33333333 -- 5.33 -- 5
E3 -- 0.33333333 -- 5.66 -- 5
E-3 -- 0.33333333 -- 6 -- 6
E3 -- 0.33333333 -- 6.33 -- 6
D3 -- 0.33333333 -- 6.66 -- 6
C3 -- 0.33333333 

G#3 -- 0.33333333 -- 5 -- 5
A3 -- 0.33333333 -- 5.33 -- 5
B3 -- 0.33333333 -- 5.66 -- 5
C4 -- 0.33333333 -- 6 -- 6
E4 -- 0.33333333 -- 6.33 -- 6
D4 -- 0.33333333 -- 6.66 -- 6
F4 -- 0.33333333 -- 7 -- 7
E4 -- 0.33333333 -- 7.33 -- 7
D4 -- 0.33333333 -- 7.66 -- 7
C4 -- 0.33333333 -- 8 -- 8
B3 -- 0.33333333 -- 8.33 -- 8
A3 -- 0.33333333 -- 8.66 -- 8
G#3 -- 0.33333333 -- 9 -- 9
B3 -- 0.33333333 -- 9.33 -- 9
A3 -- 0.33333333 -- 9.66 -- 9
C4 -- 0.33333333 -- 10 -- 10
B3 -- 0.33333333 -- 10.33 -- 10
A3 -- 0.33333333 -- 10.66 -- 10
A3 -- 0.33333333 -- 11 -- 11
G3 -- 0.33333333 -- 11.33 -- 11
F3 -- 0.33333333 -- 11.66 -- 11
E3 -- 0.33333333 -- 12 -- 12
G3 -- 0.33333333 -- 12.33 -- 12
F3 -- 0.33333333 -- 12.66 -- 12
A3 -- 0.33333333 -- 1 -- 1
G3 -- 0.33333333 -- 1.33 -- 1
F3 -- 0.33333333 -- 1.66 -- 1
E2 -- 0.5 -- 2 -- 2
E4 -- 0.5 -- 2.5 -- 2
E3 -- 0.5 -- 3 -- 3
G#3 -- 0.5 -- 3.5 -- 3
B3 -- 0.5 -- 4 -- 4
E4 -- 0.5 -- 4.5 -- 4
F2 -- 0.33333333 -- 5 -- 5
A2 -- 0.33333333 -- 5.33 -- 5
B2 -- 0.33333

E-3 -- 1.0 -- 9 -- 9
E3 -- 1.0 -- 10 -- 10
E-3 -- 0.25 -- 11 -- 11
F3 -- 0.25 -- 11.25 -- 11
G3 -- 0.25 -- 11.5 -- 11
E-3 -- 0.25 -- 11.75 -- 11
F3 -- 0.25 -- 12 -- 12
G3 -- 0.25 -- 12.25 -- 12
E-3 -- 0.25 -- 12.5 -- 12
F3 -- 0.25 -- 12.75 -- 12
G3 -- 0.25 -- 1 -- 1
G#3 -- 0.25 -- 1.25 -- 1
G3 -- 0.25 -- 1.5 -- 1
F3 -- 0.25 -- 1.75 -- 1
E3 -- 1.0 -- 2 -- 2
rest -- 2.0 -- 3 -- 3
E3 -- 1.0 -- 5 -- 5
F3 -- 1.0 -- 6 -- 6
G3 -- 1.0 -- 7 -- 7
F#3 -- 1.0 -- 8 -- 8
F3 -- 1.0 -- 9 -- 9
E3 -- 0.25 -- 10 -- 10
F3 -- 0.25 -- 10.25 -- 10
E3 -- 0.25 -- 10.5 -- 10
D3 -- 0.25 -- 10.75 -- 10
E3 -- 0.25 -- 11 -- 11
F3 -- 0.25 -- 11.25 -- 11
E3 -- 0.25 -- 11.5 -- 11
D3 -- 0.25 -- 11.75 -- 11
E3 -- 0.5 -- 12 -- 12
F3 -- 0.5 -- 12.5 -- 12
G3 -- 0.5 -- 1 -- 1
F3 -- 0.5 -- 1.5 -- 1
E3 -- 1.0 -- 2 -- 2
rest -- 2.0 -- 3 -- 3
E3 -- 0.25 -- 5 -- 5
F3 -- 0.25 -- 5.25 -- 5
E3 -- 0.25 -- 5.5 -- 5
D3 -- 0.25 -- 5.75 -- 5
C#3 -- 0.25 -- 6 -- 6
D3 -- 0.25 -- 6.25 -- 6
E3 -- 0.25 -- 6.5 -- 6
F3 -- 0.25 -- 6.75 -- 6
G#3

E4 -- 0.5 -- 12.5 -- 12
E3 -- 0.5 -- 1 -- 1
G#3 -- 0.5 -- 1.5 -- 1
B3 -- 0.5 -- 2 -- 2
E4 -- 0.5 -- 2.5 -- 2
A3 -- 0.33333333 -- 3 -- 3
B3 -- 0.33333333 -- 3.33 -- 3
C4 -- 0.33333333 -- 3.66 -- 3
D4 -- 0.33333333 -- 4 -- 4
E4 -- 0.33333333 -- 4.33 -- 4
F4 -- 0.33333333 -- 4.66 -- 4
G4 -- 0.33333333 -- 5 -- 5
F4 -- 0.33333333 -- 5.33 -- 5
E4 -- 0.33333333 -- 5.66 -- 5
F4 -- 0.33333333 -- 6 -- 6
E4 -- 0.33333333 -- 6.33 -- 6
D4 -- 0.33333333 -- 6.66 -- 6
E4 -- 0.33333333 -- 7 -- 7
D4 -- 0.33333333 -- 7.33 -- 7
C4 -- 0.33333333 -- 7.66 -- 7
D4 -- 0.33333333 -- 8 -- 8
C4 -- 0.33333333 -- 8.33 -- 8
B3 -- 0.33333333 -- 8.66 -- 8
C4 -- 0.33333333 -- 9 -- 9
B3 -- 0.33333333 -- 9.33 -- 9
A3 -- 0.33333333 -- 9.66 -- 9
B3 -- 0.33333333 -- 10 -- 10
A3 -- 0.33333333 -- 10.33 -- 10
G#3 -- 0.33333333 -- 10.66 -- 10
A3 -- 0.33333333 -- 11 -- 11
G#3 -- 0.33333333 -- 11.33 -- 11
F3 -- 0.33333333 -- 11.66 -- 11
E2 -- 0.5 -- 12 -- 12
E4 -- 0.5 -- 12.5 -- 12
E3 -- 0.5 -- 1 -- 1
G#3 -- 0.5 -- 1.5 -- 1
B3 --

In [259]:
# La lista con las notas se transforma en una lista a la que se le concatena la duración de la nota
# Es decir si la nota es G3 y su duración 0.25 (semicorchea) pasa a ser 'G3|0.25'

notas_aux=[]
for i in range(len(duraciones)):
    print ('Loop:', i, 'Nota:', notas[i], 'Duracion:',duraciones[i], 'Ritmo:', compas_flamenco[i])
    notas_aux.append(notas[i]+'|'+str(duraciones[i]))

notas=notas_aux

Loop: 0 Nota: START Duracion: 0.0 Ritmo: 1
Loop: 1 Nota: START Duracion: 0.0 Ritmo: 1
Loop: 2 Nota: START Duracion: 0.0 Ritmo: 1
Loop: 3 Nota: START Duracion: 0.0 Ritmo: 1
Loop: 4 Nota: START Duracion: 0.0 Ritmo: 1
Loop: 5 Nota: START Duracion: 0.0 Ritmo: 1
Loop: 6 Nota: START Duracion: 0.0 Ritmo: 1
Loop: 7 Nota: START Duracion: 0.0 Ritmo: 1
Loop: 8 Nota: START Duracion: 0.0 Ritmo: 1
Loop: 9 Nota: START Duracion: 0.0 Ritmo: 1
Loop: 10 Nota: START Duracion: 0.0 Ritmo: 1
Loop: 11 Nota: START Duracion: 0.0 Ritmo: 1
Loop: 12 Nota: START Duracion: 0.0 Ritmo: 1
Loop: 13 Nota: START Duracion: 0.0 Ritmo: 1
Loop: 14 Nota: START Duracion: 0.0 Ritmo: 1
Loop: 15 Nota: START Duracion: 0.0 Ritmo: 1
Loop: 16 Nota: A2 Duracion: 0.25 Ritmo: 1
Loop: 17 Nota: C4 Duracion: 0.25 Ritmo: 1
Loop: 18 Nota: E3 Duracion: 0.25 Ritmo: 1
Loop: 19 Nota: A3 Duracion: 0.25 Ritmo: 1
Loop: 20 Nota: C4 Duracion: 0.25 Ritmo: 2
Loop: 21 Nota: E3 Duracion: 0.25 Ritmo: 2
Loop: 22 Nota: D3 Duracion: 0.25 Ritmo: 2
Loop: 23 Not

Loop: 230 Nota: A3 Duracion: 0.25 Ritmo: 1
Loop: 231 Nota: E3 Duracion: 0.25 Ritmo: 2
Loop: 232 Nota: A3 Duracion: 0.25 Ritmo: 2
Loop: 233 Nota: C4 Duracion: 0.25 Ritmo: 2
Loop: 234 Nota: E4 Duracion: 0.25 Ritmo: 2
Loop: 235 Nota: D4 Duracion: 0.25 Ritmo: 3
Loop: 236 Nota: C4 Duracion: 0.25 Ritmo: 3
Loop: 237 Nota: B3 Duracion: 0.25 Ritmo: 3
Loop: 238 Nota: A3 Duracion: 0.25 Ritmo: 3
Loop: 239 Nota: G2 Duracion: 0.25 Ritmo: 4
Loop: 240 Nota: B3 Duracion: 0.25 Ritmo: 4
Loop: 241 Nota: G3 Duracion: 0.25 Ritmo: 4
Loop: 242 Nota: F3 Duracion: 0.25 Ritmo: 4
Loop: 243 Nota: B2 Duracion: 0.25 Ritmo: 5
Loop: 244 Nota: F3 Duracion: 0.25 Ritmo: 5
Loop: 245 Nota: G3 Duracion: 0.25 Ritmo: 5
Loop: 246 Nota: B3 Duracion: 0.25 Ritmo: 5
Loop: 247 Nota: C4 Duracion: 0.25 Ritmo: 6
Loop: 248 Nota: B3 Duracion: 0.25 Ritmo: 6
Loop: 249 Nota: A3 Duracion: 0.25 Ritmo: 6
Loop: 250 Nota: G3 Duracion: 0.25 Ritmo: 6
Loop: 251 Nota: F2 Duracion: 0.25 Ritmo: 7
Loop: 252 Nota: B3 Duracion: 0.25 Ritmo: 7
Loop: 253 N

Loop: 504 Nota: B4 Duracion: 0.25 Ritmo: 3
Loop: 505 Nota: C5 Duracion: 0.25 Ritmo: 3
Loop: 506 Nota: D5 Duracion: 0.25 Ritmo: 3
Loop: 507 Nota: B4 Duracion: 0.25 Ritmo: 3
Loop: 508 Nota: G3 Duracion: 0.25 Ritmo: 4
Loop: 509 Nota: B4 Duracion: 0.25 Ritmo: 4
Loop: 510 Nota: G4 Duracion: 0.25 Ritmo: 4
Loop: 511 Nota: D4 Duracion: 0.25 Ritmo: 4
Loop: 512 Nota: B3 Duracion: 0.25 Ritmo: 5
Loop: 513 Nota: D4 Duracion: 0.25 Ritmo: 5
Loop: 514 Nota: G4 Duracion: 0.25 Ritmo: 5
Loop: 515 Nota: B4 Duracion: 0.25 Ritmo: 5
Loop: 516 Nota: A4 Duracion: 0.25 Ritmo: 6
Loop: 517 Nota: B4 Duracion: 0.25 Ritmo: 6
Loop: 518 Nota: C5 Duracion: 0.25 Ritmo: 6
Loop: 519 Nota: A4 Duracion: 0.25 Ritmo: 6
Loop: 520 Nota: F3 Duracion: 0.25 Ritmo: 7
Loop: 521 Nota: A4 Duracion: 0.25 Ritmo: 7
Loop: 522 Nota: F4 Duracion: 0.25 Ritmo: 7
Loop: 523 Nota: C4 Duracion: 0.25 Ritmo: 7
Loop: 524 Nota: A3 Duracion: 0.25 Ritmo: 8
Loop: 525 Nota: C4 Duracion: 0.25 Ritmo: 8
Loop: 526 Nota: F4 Duracion: 0.25 Ritmo: 8
Loop: 527 N

Loop: 807 Nota: C3 Duracion: 0.25 Ritmo: 6
Loop: 808 Nota: D3 Duracion: 0.25 Ritmo: 7
Loop: 809 Nota: B2 Duracion: 0.25 Ritmo: 7
Loop: 810 Nota: A2 Duracion: 0.25 Ritmo: 7
Loop: 811 Nota: G2 Duracion: 0.25 Ritmo: 7
Loop: 812 Nota: F2 Duracion: 0.25 Ritmo: 8
Loop: 813 Nota: A2 Duracion: 0.25 Ritmo: 8
Loop: 814 Nota: D3 Duracion: 0.25 Ritmo: 8
Loop: 815 Nota: E3 Duracion: 0.25 Ritmo: 8
Loop: 816 Nota: D3 Duracion: 0.25 Ritmo: 9
Loop: 817 Nota: C3 Duracion: 0.25 Ritmo: 9
Loop: 818 Nota: A2 Duracion: 0.25 Ritmo: 9
Loop: 819 Nota: G#2 Duracion: 0.25 Ritmo: 9
Loop: 820 Nota: A2 Duracion: 0.25 Ritmo: 10
Loop: 821 Nota: F2 Duracion: 0.25 Ritmo: 10
Loop: 822 Nota: G2 Duracion: 0.25 Ritmo: 10
Loop: 823 Nota: F2 Duracion: 0.25 Ritmo: 10
Loop: 824 Nota: E2 Duracion: 0.5 Ritmo: 11
Loop: 825 Nota: E4 Duracion: 0.5 Ritmo: 11
Loop: 826 Nota: E3 Duracion: 0.5 Ritmo: 12
Loop: 827 Nota: G#3 Duracion: 0.5 Ritmo: 12
Loop: 828 Nota: B3 Duracion: 0.5 Ritmo: 1
Loop: 829 Nota: E4 Duracion: 0.5 Ritmo: 1
Loop: 8

Loop: 1134 Nota: START Duracion: 0.0 Ritmo: 2
Loop: 1135 Nota: START Duracion: 0.0 Ritmo: 2
Loop: 1136 Nota: START Duracion: 0.0 Ritmo: 2
Loop: 1137 Nota: START Duracion: 0.0 Ritmo: 2
Loop: 1138 Nota: START Duracion: 0.0 Ritmo: 2
Loop: 1139 Nota: START Duracion: 0.0 Ritmo: 2
Loop: 1140 Nota: START Duracion: 0.0 Ritmo: 2
Loop: 1141 Nota: G4 Duracion: 0.25 Ritmo: 2
Loop: 1142 Nota: F4 Duracion: 0.25 Ritmo: 2
Loop: 1143 Nota: E4 Duracion: 0.25 Ritmo: 2
Loop: 1144 Nota: D4 Duracion: 0.25 Ritmo: 2
Loop: 1145 Nota: F4 Duracion: 0.25 Ritmo: 3
Loop: 1146 Nota: E4 Duracion: 0.25 Ritmo: 3
Loop: 1147 Nota: D4 Duracion: 0.25 Ritmo: 3
Loop: 1148 Nota: C4 Duracion: 0.25 Ritmo: 3
Loop: 1149 Nota: E4 Duracion: 0.25 Ritmo: 4
Loop: 1150 Nota: D4 Duracion: 0.25 Ritmo: 4
Loop: 1151 Nota: C4 Duracion: 0.25 Ritmo: 4
Loop: 1152 Nota: B3 Duracion: 0.25 Ritmo: 4
Loop: 1153 Nota: D4 Duracion: 0.25 Ritmo: 5
Loop: 1154 Nota: C4 Duracion: 0.25 Ritmo: 5
Loop: 1155 Nota: B3 Duracion: 0.25 Ritmo: 5
Loop: 1156 Nota: A

...

Loop: 9478 Nota: E2 Duracion: 0.5 Ritmo: 8
Loop: 9479 Nota: E4 Duracion: 0.5 Ritmo: 8
Loop: 9480 Nota: E3 Duracion: 0.5 Ritmo: 9
Loop: 9481 Nota: G#3 Duracion: 0.5 Ritmo: 9
Loop: 9482 Nota: E4 Duracion: 1.0 Ritmo: 10
Loop: 9483 Nota: A2 Duracion: 0.33333333 Ritmo: 11
Loop: 9484 Nota: F2 Duracion: 0.33333333 Ritmo: 11
Loop: 9485 Nota: A2 Duracion: 0.33333333 Ritmo: 11
Loop: 9486 Nota: B2 Duracion: 0.33333333 Ritmo: 12
Loop: 9487 Nota: C3 Duracion: 0.33333333 Ritmo: 12
Loop: 9488 Nota: D3 Duracion: 0.33333333 Ritmo: 12
Loop: 9489 Nota: E-3 Duracion: 0.33333333 Ritmo: 1
Loop: 9490 Nota: E3 Duracion: 0.33333333 Ritmo: 1
Loop: 9491 Nota: F3 Duracion: 0.33333333 Ritmo: 1
Loop: 9492 Nota: E2 Duracion: 0.5 Ritmo: 2
Loop: 9493 Nota: E4 Duracion: 0.5 Ritmo: 2
Loop: 9494 Nota: E3 Duracion: 0.5 Ritmo: 3
Loop: 9495 Nota: G#3 Duracion: 0.5 Ritmo: 3
Loop: 9496 Nota: E4 Duracion: 1.0 Ritmo: 4
Loop: 9497 Nota: A2 Duracion: 0.33333333 Ritmo: 5
Loop: 9498 Nota: F2 Duracion: 0.33333333 Ritmo: 5
Loop: 9499

Loop: 9791 Nota: G#2 Duracion: 0.33333333 Ritmo: 5
Loop: 9792 Nota: A2 Duracion: 0.33333333 Ritmo: 5
Loop: 9793 Nota: B2 Duracion: 0.33333333 Ritmo: 6
Loop: 9794 Nota: C3 Duracion: 0.33333333 Ritmo: 6
Loop: 9795 Nota: D3 Duracion: 0.33333333 Ritmo: 6
Loop: 9796 Nota: E3 Duracion: 0.33333333 Ritmo: 7
Loop: 9797 Nota: D3 Duracion: 0.33333333 Ritmo: 7
Loop: 9798 Nota: C3 Duracion: 0.33333333 Ritmo: 7
Loop: 9799 Nota: B2 Duracion: 0.33333333 Ritmo: 8
Loop: 9800 Nota: C3 Duracion: 0.33333333 Ritmo: 8
Loop: 9801 Nota: D3 Duracion: 0.33333333 Ritmo: 8
Loop: 9802 Nota: E3 Duracion: 0.33333333 Ritmo: 9
Loop: 9803 Nota: F3 Duracion: 0.33333333 Ritmo: 9
Loop: 9804 Nota: G3 Duracion: 0.33333333 Ritmo: 9
Loop: 9805 Nota: A3 Duracion: 0.33333333 Ritmo: 10
Loop: 9806 Nota: G3 Duracion: 0.33333333 Ritmo: 10
Loop: 9807 Nota: F3 Duracion: 0.33333333 Ritmo: 10
Loop: 9808 Nota: D4 Duracion: 0.33333333 Ritmo: 11
Loop: 9809 Nota: C4 Duracion: 0.33333333 Ritmo: 11
Loop: 9810 Nota: B3 Duracion: 0.33333333 Rit

Loop: 10165 Nota: E3 Duracion: 0.33333333 Ritmo: 9
Loop: 10166 Nota: A3 Duracion: 0.33333333 Ritmo: 9
Loop: 10167 Nota: B3 Duracion: 1.0 Ritmo: 10
Loop: 10168 Nota: F2 Duracion: 0.33333333 Ritmo: 11
Loop: 10169 Nota: C3 Duracion: 0.33333333 Ritmo: 11
Loop: 10170 Nota: D3 Duracion: 0.33333333 Ritmo: 11
Loop: 10171 Nota: E3 Duracion: 0.33333333 Ritmo: 12
Loop: 10172 Nota: F3 Duracion: 0.33333333 Ritmo: 12
Loop: 10173 Nota: G3 Duracion: 0.33333333 Ritmo: 12
Loop: 10174 Nota: A3 Duracion: 0.33333333 Ritmo: 1
Loop: 10175 Nota: G3 Duracion: 0.33333333 Ritmo: 1
Loop: 10176 Nota: F3 Duracion: 0.33333333 Ritmo: 1
Loop: 10177 Nota: E2 Duracion: 0.5 Ritmo: 2
Loop: 10178 Nota: E4 Duracion: 0.5 Ritmo: 2
Loop: 10179 Nota: E3 Duracion: 0.5 Ritmo: 3
Loop: 10180 Nota: G#3 Duracion: 0.5 Ritmo: 3
Loop: 10181 Nota: B3 Duracion: 0.5 Ritmo: 4
Loop: 10182 Nota: E4 Duracion: 0.5 Ritmo: 4
Loop: 10183 Nota: E3 Duracion: 0.25 Ritmo: 5
Loop: 10184 Nota: E4 Duracion: 0.25 Ritmo: 5
Loop: 10185 Nota: C4 Duracion: 0.

Loop: 10525 Nota: C3 Duracion: 0.33333333 Ritmo: 12
Loop: 10526 Nota: E-3 Duracion: 0.33333333 Ritmo: 12
Loop: 10527 Nota: C3 Duracion: 0.33333333 Ritmo: 12
Loop: 10528 Nota: A2 Duracion: 0.33333333 Ritmo: 1
Loop: 10529 Nota: G2 Duracion: 0.33333333 Ritmo: 1
Loop: 10530 Nota: F2 Duracion: 0.33333333 Ritmo: 1
Loop: 10531 Nota: E2 Duracion: 0.5 Ritmo: 2
Loop: 10532 Nota: E4 Duracion: 0.5 Ritmo: 2
Loop: 10533 Nota: G#3 Duracion: 0.5 Ritmo: 3
Loop: 10534 Nota: B3 Duracion: 0.5 Ritmo: 3
Loop: 10535 Nota: E4 Duracion: 1.0 Ritmo: 4
Loop: 10536 Nota: D3 Duracion: 0.33333333 Ritmo: 5
Loop: 10537 Nota: F3 Duracion: 0.33333333 Ritmo: 5
Loop: 10538 Nota: B3 Duracion: 0.33333333 Ritmo: 5
Loop: 10539 Nota: D3 Duracion: 0.33333333 Ritmo: 6
Loop: 10540 Nota: F3 Duracion: 0.33333333 Ritmo: 6
Loop: 10541 Nota: B3 Duracion: 0.33333333 Ritmo: 6
Loop: 10542 Nota: C4 Duracion: 0.5 Ritmo: 7
Loop: 10543 Nota: B3 Duracion: 0.5 Ritmo: 7
Loop: 10544 Nota: C3 Duracion: 0.33333333 Ritmo: 8
Loop: 10545 Nota: E3 Dur

Loop: 10852 Nota: F4 Duracion: 0.25 Ritmo: 7
Loop: 10853 Nota: E4 Duracion: 0.25 Ritmo: 7
Loop: 10854 Nota: D4 Duracion: 0.25 Ritmo: 7
Loop: 10855 Nota: C4 Duracion: 0.25 Ritmo: 7
Loop: 10856 Nota: E4 Duracion: 0.25 Ritmo: 8
Loop: 10857 Nota: D4 Duracion: 0.25 Ritmo: 8
Loop: 10858 Nota: C4 Duracion: 0.25 Ritmo: 8
Loop: 10859 Nota: B3 Duracion: 0.25 Ritmo: 8
Loop: 10860 Nota: D4 Duracion: 0.25 Ritmo: 9
Loop: 10861 Nota: C4 Duracion: 0.25 Ritmo: 9
Loop: 10862 Nota: B3 Duracion: 0.25 Ritmo: 9
Loop: 10863 Nota: A3 Duracion: 0.25 Ritmo: 9
Loop: 10864 Nota: C4 Duracion: 0.25 Ritmo: 10
Loop: 10865 Nota: B3 Duracion: 0.25 Ritmo: 10
Loop: 10866 Nota: A3 Duracion: 0.25 Ritmo: 10
Loop: 10867 Nota: G#3 Duracion: 0.25 Ritmo: 10
Loop: 10868 Nota: B3 Duracion: 0.25 Ritmo: 11
Loop: 10869 Nota: A3 Duracion: 0.25 Ritmo: 11
Loop: 10870 Nota: G#3 Duracion: 0.25 Ritmo: 11
Loop: 10871 Nota: F3 Duracion: 0.25 Ritmo: 11
Loop: 10872 Nota: E2 Duracion: 0.5 Ritmo: 12
Loop: 10873 Nota: E4 Duracion: 0.5 Ritmo: 12


In [260]:
notas

['START|0.0',
 'START|0.0',
 'START|0.0',
 'START|0.0',
 'START|0.0',
 'START|0.0',
 'START|0.0',
 'START|0.0',
 'START|0.0',
 'START|0.0',
 'START|0.0',
 'START|0.0',
 'START|0.0',
 'START|0.0',
 'START|0.0',
 'START|0.0',
 'A2|0.25',
 'C4|0.25',
 'E3|0.25',
 'A3|0.25',
 'C4|0.25',
 'E3|0.25',
 'D3|0.25',
 'C3|0.25',
 'D3|0.25',
 'C3|0.25',
 'B2|0.25',
 'A2|0.25',
 'G2|0.25',
 'B3|0.25',
 'B2|0.25',
 'D3|0.25',
 'G3|0.25',
 'E3|0.25',
 'D3|0.25',
 'C3|0.25',
 'D3|0.25',
 'B2|0.25',
 'A2|0.25',
 'G2|0.25',
 'F2|0.25',
 'A2|0.25',
 'D3|0.25',
 'E3|0.25',
 'D3|0.25',
 'C3|0.25',
 'A2|0.25',
 'G#2|0.25',
 'A2|0.25',
 'F2|0.25',
 'G2|0.25',
 'F2|0.25',
 'E2|0.5',
 'E4|0.5',
 'E3|0.5',
 'G#3|0.5',
 'B3|0.5',
 'E4|0.5',
 'A2|0.25',
 'C4|0.25',
 'E3|0.25',
 'A3|0.25',
 'C4|0.25',
 'E3|0.25',
 'D3|0.25',
 'C3|0.25',
 'D3|0.25',
 'C3|0.25',
 'B2|0.25',
 'A2|0.25',
 'G2|0.25',
 'B3|0.25',
 'B2|0.25',
 'D3|0.25',
 'G3|0.25',
 'E3|0.25',
 'D3|0.25',
 'C3|0.25',
 'D3|0.25',
 'B2|0.25',
 'A2|0.25',


In [261]:
# Obtiene todos los nombres diferentes de las notas ordenadas y el numero de todas ellas. Devuelve un lista con las
# notas diferentes
def recuperar_elementos_diferentes(elementos):
    nombre_elementos = sorted(set(elementos))
    numero_elementos = len(nombre_elementos)
    return (nombre_elementos, numero_elementos)

In [262]:
# Creamos un diccionario donde se mapea cada nota con su correspondiente duracion a un valor entero
def crear_diccionarios(nombre_elementos):
    elemento_to_entero = dict((elemento, numero) for numero, elemento in enumerate(nombre_elementos))
    entero_to_elemento = dict((numero, elemento) for numero, elemento in enumerate(nombre_elementos))

    return (elemento_to_entero, entero_to_elemento)

In [263]:
compas_flamenco_nombre, compas_flamenco_numero = recuperar_elementos_diferentes(compas_flamenco)
#print (compas_flamenco_nombre, compas_flamenco_numero)

In [264]:
# Recoge todas las notas, duraciones y compas que son diferentes
notas_nombre, notas_numero = recuperar_elementos_diferentes(notas)
duraciones_nombre, duraciones_numero = recuperar_elementos_diferentes(duraciones)
compas_flamenco_nombre, compas_flamenco_numero = recuperar_elementos_diferentes(compas_flamenco)

elementos_distintos = [notas_nombre, notas_numero, duraciones_nombre, duraciones_numero, compas_flamenco_nombre, compas_flamenco_numero]

with open(os.path.join(directorio_almacenamiento, 'elementos_distintos'), 'wb') as f:
    pickle.dump(elementos_distintos, f)

# Construye un diccionario con el nombre de las notas y un entero y viceversa
nota_to_entero, entero_to_nota = crear_diccionarios(notas_nombre)

# Construye un diccionario con la duracion de las notas y un entero y viceversa
duracion_to_entero, entero_to_duracion = crear_diccionarios(duraciones_nombre)

# Construye un diccionario con el valor del compas un entero y viceversa
compas_flamenco_to_entero, entero_to_compas_flamenco = crear_diccionarios(compas_flamenco_nombre)

diccionarios_notas_duraciones = [nota_to_entero, entero_to_nota, duracion_to_entero, entero_to_duracion, compas_flamenco_to_entero, entero_to_compas_flamenco]

with open(os.path.join(directorio_almacenamiento, 'diccionarios_notas_duraciones'), 'wb') as f:
    pickle.dump(diccionarios_notas_duraciones, f)

In [265]:
# Contiene 4 listas siendo cada una de ellas u diccionario
diccionarios_notas_duraciones

[{'A2|0.25': 0,
  'A2|0.33333333': 1,
  'A2|0.5': 2,
  'A2|1.0': 3,
  'A3|0.25': 4,
  'A3|0.33333333': 5,
  'A3|0.5': 6,
  'A3|0.75': 7,
  'A3|1.0': 8,
  'A4|0.25': 9,
  'A4|0.33333333': 10,
  'A4|0.5': 11,
  'A4|0.66666667': 12,
  'B-2|0.25': 13,
  'B-2|0.33333333': 14,
  'B-2|0.5': 15,
  'B-3|0.25': 16,
  'B-3|0.33333333': 17,
  'B-3|0.5': 18,
  'B-4|0.25': 19,
  'B2|0.25': 20,
  'B2|0.33333333': 21,
  'B2|0.5': 22,
  'B2|0.75': 23,
  'B2|1.0': 24,
  'B3|0.25': 25,
  'B3|0.33333333': 26,
  'B3|0.5': 27,
  'B3|0.75': 28,
  'B3|1.0': 29,
  'B3|3.0': 30,
  'B4|0.25': 31,
  'B4|0.33333333': 32,
  'B4|0.5': 33,
  'B4|1.0': 34,
  'C#3|0.25': 35,
  'C#3|0.33333333': 36,
  'C#3|0.5': 37,
  'C#4|0.25': 38,
  'C#4|0.33333333': 39,
  'C#4|0.5': 40,
  'C#4|1.0': 41,
  'C#5|0.25': 42,
  'C#5|0.5': 43,
  'C3|0.25': 44,
  'C3|0.33333333': 45,
  'C3|0.5': 46,
  'C3|1.0': 47,
  'C4|0.25': 48,
  'C4|0.33333333': 49,
  'C4|0.5': 50,
  'C4|1.0': 51,
  'C5|0.25': 52,
  'C5|0.33333333': 53,
  'C5|1.0': 54

In [266]:
# Divide las partituras en secuencias de longitud longitud_secuencia. Estas serán usadas para entrenar la red
# Retorna 2 listas:
# 1- Contiene una lista con 2 sublistas:
#    1.1- Lista con todas las secuencias de notas
#    1.2- Lista con todas las secuencias de duracion
# 2- Contiene una lista con la salida siguiente a la secuencia de entrada. A su vez tiene 2 sublista
#    2.1- Lista con la nota siguiente
#    2.2- Lista con la duración siguiente

def crear_secuencias(notas, duraciones, compas_flamenco, diccionarios_notas_duraciones, elementos_distintos, longitud_secuencia =32):

    # Se crean los diccionarios que relacionan los elementos con las categoria
    nota_to_entero, entero_to_nota, duracion_to_entero, entero_to_duracion, compas_flamenco_to_entero, entero_to_compas_flamenco = diccionarios_notas_duraciones
    notas_nombre, notas_numero, duraciones_nombre, duraciones_numero, flamenco_compas_names, compas_flamenco_numero = elementos_distintos

    # Inicializa las listas a vacio
    notas_red_neuronal_entrada = []
    notas_red_neuronal_salida = []
    duraciones_red_neuronal_entrada = []
    duraciones_red_neuronal_salida = []
    compas_flamenco_red_neuronal_entrada=[]
    compas_flamenco_red_neuronal_salida=[]

    # crea las diferentes secuencias de entrada y su correspondiente salida
    for i in range(len(notas) - longitud_secuencia):
        # División en secuencias para las notas
        notas_secuencia_entrada = notas[i:i + longitud_secuencia]
        notas_secuencia_salida = notas[i + longitud_secuencia]
        notas_red_neuronal_entrada.append([nota_to_entero[char] for char in notas_secuencia_entrada])
        notas_red_neuronal_salida.append(nota_to_entero[notas_secuencia_salida])

        # División en secuencias para las duraciones
        duraciones_secuencia_entrada = duraciones[i:i + longitud_secuencia]
        duraciones_secuencia_salida = duraciones[i + longitud_secuencia]
        duraciones_red_neuronal_entrada.append([duracion_to_entero[char] for char in duraciones_secuencia_entrada])
        duraciones_red_neuronal_salida.append(duracion_to_entero[duraciones_secuencia_salida])

        # División en secuencias para el compas
        compas_flamenco_secuencia_entrada = compas_flamenco[i:i + longitud_secuencia]
        compas_flamenco_secuencia_salida = compas_flamenco[i + longitud_secuencia]
        compas_flamenco_red_neuronal_entrada.append([compas_flamenco_to_entero[char] for char in compas_flamenco_secuencia_entrada])
        compas_flamenco_red_neuronal_salida.append(compas_flamenco_to_entero[compas_flamenco_secuencia_salida])
        
        
    numero_secuencias_creadas = len(notas_red_neuronal_entrada)
    print('Numero_secuencias_creadas:',numero_secuencias_creadas)

    # Reformatea la entrada a la red a un formato compatible con la LSTM
    notas_red_neuronal_entrada = np.reshape(notas_red_neuronal_entrada, (numero_secuencias_creadas, longitud_secuencia))
    duraciones_red_neuronal_entrada = np.reshape(duraciones_red_neuronal_entrada, (numero_secuencias_creadas, longitud_secuencia))
    compas_flamenco_red_neuronal_entrada = np.reshape(compas_flamenco_red_neuronal_entrada, (numero_secuencias_creadas, longitud_secuencia))

    red_neuronal_entrada = [notas_red_neuronal_entrada, duraciones_red_neuronal_entrada, compas_flamenco_red_neuronal_entrada]

    notas_red_neuronal_salida = np_utils.to_categorical(notas_red_neuronal_salida, num_classes=notas_numero)
    duraciones_red_neuronal_salida = np_utils.to_categorical(duraciones_red_neuronal_salida, num_classes=duraciones_numero)
    compas_flamenco_red_neuronal_salida = np_utils.to_categorical(compas_flamenco_red_neuronal_salida, num_classes=compas_flamenco_numero)
    
    red_neuronal_salida = [notas_red_neuronal_salida, duraciones_red_neuronal_salida, compas_flamenco_red_neuronal_salida]

    return (red_neuronal_entrada, red_neuronal_salida)

In [267]:
# red_neuronal_entrada es un lista de listas:
# 1ra lista  red_neuronal_entrada[0] contiene las diferentes secuencias de longitud n de notas+duraciones codificadas
# 2da lista  red_neuronal_entrada[1] contiene las diferentes secuencias de longitud n de las duraciones codificadas
# 3ra lista  red_neuronal_entrada[2] contiene las diferentes secuencias de longitud n de compas

red_neuronal_entrada, red_neuronal_salida = crear_secuencias(notas, duraciones, compas_flamenco, diccionarios_notas_duraciones, elementos_distintos, longitud_secuencia)

Numero_secuencias_creadas: 10910


In [268]:
# Creacion de la estructura de la red neuronal


def crear_red_neuronal(notas_numero, duraciones_numero, compas_flamenco_numero, tamanio_embedding = 100, neuronas_LSTM = 256, usar_attention = False):
 
    notas_in = Input(shape = (None,))
    duraciones_in = Input(shape = (None,))
    compas_flamenco_in = Input(shape = (None,))

    x1 = Embedding(notas_numero, tamanio_embedding)(notas_in)
    x2 = Embedding(duraciones_numero, tamanio_embedding)(duraciones_in) 
    x3 = Embedding(compas_flamenco_numero, tamanio_embedding)(compas_flamenco_in) 

    x = Concatenate()([x1,x2,x3])

    x = Bidirectional(LSTM(neuronas_LSTM, return_sequences=True))(x)
    x = Dropout(0.2)(x)

    # Si está activo utiliza el mecanismo de attention
    if usar_attention:
        x = LSTM(neuronas_LSTM, return_sequences=True)(x)
        x = Dropout(0.2)(x)
        e = Dense(1, activation='tanh')(x)
        e = Reshape([-1])(e)

        alpha = Activation('softmax')(e)
        alpha_repeated = Permute([2, 1])(RepeatVector(neuronas_LSTM)(alpha))

        c = Multiply()([x, alpha_repeated])
        c = Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(neuronas_LSTM,))(c)
    
    else:
        c = LSTM(neuronas_LSTM)(x)
        c = Dropout(0.2)(c)
                                    
    notas_out = Dense(notas_numero, activation = 'softmax', name = 'notas_y_duracion')(c)
    duraciones_out = Dense(duraciones_numero, activation = 'softmax', name = 'duracion')(c)
    compas_flamenco_out = Dense(compas_flamenco_numero, activation = 'softmax', name = 'compas_flamenco')(c)
   
    model = Model([notas_in, duraciones_in, compas_flamenco_in], [notas_out, duraciones_out, compas_flamenco_out])
    

    if usar_attention:
        modelo_attention = Model([notas_in, duraciones_in, compas_flamenco_in], alpha)
    else:
        modelo_attention = None


    opti = RMSprop(lr = 0.001)
    model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy',  'categorical_crossentropy'], optimizer=opti, metrics = ['accuracy'])

    return model, modelo_attention


In [269]:
model, modelo_attention = crear_red_neuronal(notas_numero, duraciones_numero, compas_flamenco_numero, tamanio_embedding, neuronas_LSTM, usar_attention)
model.summary()

Model: "functional_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_18 (Embedding)        (None, None, 128)    21120       input_19[0][0]                   
______________________________________________________________________________________

In [270]:
# Definimos el directorio para almacenar los pesos
#model.load_weights(os.path.join(directorio_pesos, "pesos.valores"))
directorio_pesos = os.path.join(directorio_ejecucion, 'pesos')

In [271]:
directorio_pesos

'./ejecucion/composicion/0001_guitarra\\pesos'

In [272]:
# Guardamos los pesos de la red segun se va entrenando
# en checkoint1 tenemos las ponderaciones parciales para las diferentes epoch
checkpoint1 = ModelCheckpoint(os.path.join(directorio_pesos, "pesos-{epoch:02d}-{loss:.4f}-bigger.h5"),
                              monitor='loss', verbose=0, save_best_only=True, mode='min')

# En checkpoint2 tenemos las ultimas ponderaciones calculadas
checkpoint2 = ModelCheckpoint(os.path.join(directorio_pesos, "pesos.h5"),
                              monitor='loss', verbose=0, save_best_only=True, mode='min')

early_stopping = EarlyStopping(monitor='loss', restore_best_weights=True, patience = 3)

callbacks_list = [checkpoint1, checkpoint2, early_stopping]

model.save_weights(os.path.join(directorio_pesos, "pesos.h5"))

# Training del modelo
model.fit(red_neuronal_entrada, red_neuronal_salida, 
          epochs=2000000, batch_size=32, callbacks=callbacks_list, shuffle=True )


Epoch 1/2000000
341/341 [==============================] - 28s 83ms/step - loss: 5.3016 - notas_y_duracion_loss: 3.6848 - duracion_loss: 0.6345 - compas_flamenco_loss: 0.9823 - notas_y_duracion_accuracy: 0.1269 - duracion_accuracy: 0.8236 - compas_flamenco_accuracy: 0.6299
Epoch 2/2000000
341/341 [==============================] - 30s 89ms/step - loss: 4.2372 - notas_y_duracion_loss: 3.3318 - duracion_loss: 0.4693 - compas_flamenco_loss: 0.4361 - notas_y_duracion_accuracy: 0.1592 - duracion_accuracy: 0.8710 - compas_flamenco_accuracy: 0.8295
Epoch 3/2000000
341/341 [==============================] - 30s 88ms/step - loss: 3.8814 - notas_y_duracion_loss: 3.1851 - duracion_loss: 0.4175 - compas_flamenco_loss: 0.2788 - notas_y_duracion_accuracy: 0.1786 - duracion_accuracy: 0.8810 - compas_flamenco_accuracy: 0.8937
Epoch 4/2000000
341/341 [==============================] - 32s 94ms/step - loss: 3.6334 - notas_y_duracion_loss: 3.0513 - duracion_loss: 0.3861 - compas_flamenco_loss: 0.1960 - n

341/341 [==============================] - 62s 183ms/step - loss: 0.5069 - notas_y_duracion_loss: 0.4390 - duracion_loss: 0.0448 - compas_flamenco_loss: 0.0231 - notas_y_duracion_accuracy: 0.9106 - duracion_accuracy: 0.9860 - compas_flamenco_accuracy: 0.9924
Epoch 59/2000000
341/341 [==============================] - 61s 179ms/step - loss: 0.4978 - notas_y_duracion_loss: 0.4307 - duracion_loss: 0.0468 - compas_flamenco_loss: 0.0203 - notas_y_duracion_accuracy: 0.9126 - duracion_accuracy: 0.9857 - compas_flamenco_accuracy: 0.9939
Epoch 60/2000000
341/341 [==============================] - 61s 178ms/step - loss: 0.4869 - notas_y_duracion_loss: 0.4211 - duracion_loss: 0.0420 - compas_flamenco_loss: 0.0238 - notas_y_duracion_accuracy: 0.9134 - duracion_accuracy: 0.9876 - compas_flamenco_accuracy: 0.9924
Epoch 61/2000000
341/341 [==============================] - 63s 183ms/step - loss: 0.4689 - notas_y_duracion_loss: 0.4061 - duracion_loss: 0.0440 - compas_flamenco_loss: 0.0188 - notas_y_du

In [273]:
######################### PREDICCION ###############################

In [274]:
# Definimos el directorio para almacenar los pesos
directorio_pesos = os.path.join(directorio_ejecucion, 'pesos')

# Cargamos los pesos del modelo entrenado
model.load_weights(os.path.join(directorio_pesos, "pesos.h5"))

In [723]:
#####################################
# Parametros para la predicción
#####################################
#notas_temp=0.5
#duracion_temp = 0.5
notas_temp= 1.5
duracion_temp = 0
compas_flamenco_temp=0
#####################################
#max_extra_notas = 1000
#max_longitud_secuencia = 32
#longitud_secuencia = 32
max_extra_notas = 500 # EL numero total de notas que se deben generar
max_longitud_secuencia = 16 # Longitud de la secuencia que vamos a generar 
longitud_secuencia = 16 # Longitud de la secuencia que vamos a procesar. 
                        #Este valor se compará con el max_longitud_secuencia en la prediccion para generar las nots
#####################################

#notas = ['START', 'D3', 'D3', 'E3', 'D3', 'G3', 'F#3','D3', 'D3', 'E3', 'D3', 'G3', 'F#3','D3', 'D3', 'E3', 'D3', 'G3', 'F#3','D3', 'D3', 'E3', 'D3', 'G3', 'F#3']
#duraciones = [0, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2]
#notas = ['START']
#duraciones = [0]

#####################################
# Secuencia inicial de entrada
#####################################
#1
#notas = ['A3|0.5', 'E4|0.5', 'C4|0.5']
#duraciones = [0.5, 0.5, 0.5]
#compas_flamenco=[1, 1, 2]

#2
#notas = ['A4|0.25', 'E4|0.25', 'C4|0.25', 'A4|0.25']
#duraciones = [0.25, 0.25, 0.25, 0.25]
#compas_flamenco=[1, 1, 1, 1]

#3
#notas = ['F3|0.25', 'B3|0.25', 'A3|0.25', 'D3|0.25']
#duraciones = [0.25, 0.25, 0.25, 0.25]
#compas_flamenco=[1, 1, 1, 1]

#4
#notas = ['C4|0.25', 'B4|0.25', 'A4|0.25', 'B4|0.25']
#duraciones = [0.25, 0.25, 0.25, 0.25]
#compas_flamenco=[1, 1, 1, 1]

#5
#notas = ['C5|0.25', 'F5|0.25', 'C5|0.25', 'F5|0.25']
#duraciones = [0.25, 0.25, 0.25, 0.25]
#compas_flamenco=[1, 1, 1, 1]

#6
#notas = ['A4|0.25', 'C5|0.25', 'F5|0.25']
#duraciones = [0.25, 0.25, 0.25]
#compas_flamenco=[1, 1, 1]

#7
#notas = ['A4|0.25', 'G4|0.25', 'A4|0.25', 'D4|0.25']
#duraciones = [0.25, 0.25, 0.25, 0.25]
#compas_flamenco=[1, 1, 1, 1]

#8
#notas = ['E4|0.5', 'F4|0.5', 'E3|0.5','E4|0.5']
#duraciones = [0.5, 0.5, 0.5, 0.5]
#compas_flamenco=[1, 1, 2, 2]

#9
#notas = ['G#3|0.5','B3|0.5']
#duraciones = [ 0.5, 0.5]
#compas_flamenco=[1, 1]

#10    
notas = ['F3|0.25']
duraciones = [0.25]
compas_flamenco=[1]
#####################################

# Si la entrada en menor que longitud_secuencia, rellena la entrada con 'START'
# para lograr una secuencia de longitud longitud_secuencia
if longitud_secuencia is not None:
    notas = ['START|0.0'] * (longitud_secuencia - len(notas)) + notas
    duraciones = [0] * (longitud_secuencia - len(duraciones)) + duraciones
    compas_flamenco= [1] * (longitud_secuencia - len(compas_flamenco)) + compas_flamenco

In [724]:
# Definicion del mecanismo de temperatura
# random.choice escoge el elemento segun las probabilidaddes de p
# Cuando la temperatura es 0 se devuelve el valor de mayor probabilidad del vector de probabilidades de la capa softmax

def mecanismo_temperatura(predicciones_softmax, temperatura):

    if temperatura == 0:
        return np.argmax(predicciones_softmax), np.argmax(predicciones_softmax)
    else:
        predicciones_softmax = np.log(predicciones_softmax) / temperatura
        exp_predicciones = np.exp(predicciones_softmax)
        predicciones_softmax = exp_predicciones / np.sum(exp_predicciones)
        return np.random.choice(len(predicciones_softmax), p=predicciones_softmax), np.argmax(predicciones_softmax)

In [725]:
prediccion_salida = []
print('prediccion_salida_1:', prediccion_salida)
notas_entrada_sequence = []
duraciones_entrada_sequence = []
compas_flamenco_entrada_sequence=[]
overall_preds = []

# notas contiene la secuencia inicial de notas a partir de la cual vamos a predecir
# duraciones contiene la secuencia inicial de duraciones a partir de la cual vamos a predecir
# Ambas la convertimos 


for n, d, c in zip(notas, duraciones, compas_flamenco):
    
    note_int = nota_to_entero[n]
    duracion_int = duracion_to_entero[d]
    compas_flamenco_int= compas_flamenco_to_entero[c]
    
    notas_entrada_sequence.append(note_int)
    duraciones_entrada_sequence.append(duracion_int)
    compas_flamenco_entrada_sequence.append(compas_flamenco_int)
    
    
    prediccion_salida.append([n, d, c])
    #print('prediccion_salida_2:', prediccion_salida)
    
    
    n_aux, d_aux= n.split("|")
    n, d = n_aux, d_aux
    
    ##########################################
    # Revisar si se puede quitar esto de abajo
    ##########################################
    if n != 'START':
        print('n:',n)
        midi_note = note.Note(n)

        new_note = np.zeros(128)
        new_note[midi_note.pitch.midi] = 1
        overall_preds.append(new_note)
        
        #print('n:',n,'--midi_note:', midi_note,'--midi_note.pitch.midi:',midi_note.pitch.midi,'--new_note:',new_note, '--overall_preds:', overall_preds)

prediccion_salida_1: []
n: F3


In [726]:
prediccion_salida

[['START|0.0', 0, 1],
 ['START|0.0', 0, 1],
 ['START|0.0', 0, 1],
 ['START|0.0', 0, 1],
 ['START|0.0', 0, 1],
 ['START|0.0', 0, 1],
 ['START|0.0', 0, 1],
 ['START|0.0', 0, 1],
 ['START|0.0', 0, 1],
 ['START|0.0', 0, 1],
 ['START|0.0', 0, 1],
 ['START|0.0', 0, 1],
 ['START|0.0', 0, 1],
 ['START|0.0', 0, 1],
 ['START|0.0', 0, 1],
 ['F3|0.25', 0.25, 1]]

In [727]:
def ajuste_temperatura_variable(i1_tv, i1_t0, compas_flamenco_vector, notas_entrada_sequence, compas_corte):

    # Convertimos de entero a nota la nota generada con la predicción
    nota_resultado_tv = entero_to_nota[i1_tv]
    nota_aux, duracion_aux= nota_resultado_tv.split("|")
    duracion_resultado_tv= duracion_aux
    
    nota_resultado_t0 = entero_to_nota[i1_t0]
    nota_aux, duracion_aux= nota_resultado_t0.split("|")
    duracion_resultado_t0= duracion_aux
    
    # Toma la ultima nota de entrada para calcular el compas
    nota_entrada = entero_to_nota[notas_entrada_sequence[-1]]
    nota_entrada_aux_valor, nota_entrada_aux_duracion=nota_entrada.split('|')
    
    acumulador_aux_tv = float(nota_entrada_aux_duracion) + compas_flamenco_vector[-1]
    acumulador_aux_tv = round(acumulador_aux_tv,8)
    # En los casos de tresillos, cinquillos, seisillos, etc ajustamos el compas por los decimales
    if (int(acumulador_aux_tv + 0.01) == round(acumulador_aux_tv, 1)):
        acumulador_aux_tv=int(acumulador_aux_tv + 0.01)
    if (acumulador_aux_tv >= 12 + 1):
        acumulador_aux_tv= acumulador_aux_tv - 12

    # Se hace lo mimo con t0
    acumulador_aux_t0 = float(nota_entrada_aux_duracion) + compas_flamenco_vector[-1]
    acumulador_aux_t0 = round(acumulador_aux_t0, 8)
    # En los casos de tresillos, cinquillos, seisillos, etc ajustamos el compas por los decimales
    if (int(acumulador_aux_t0 + 0.01) == round(acumulador_aux_t0, 1)):
        acumulador_aux_t0=int(acumulador_aux_t0 + 0.01)
    if (acumulador_aux_t0 >= 12 + 1):
        acumulador_aux_t0= acumulador_aux_t0 - 12
    
    ########################
    
    # Machacamos la duracion con el valor seleccionado de nota+duracion
    i2_tv = duracion_to_entero[float(duracion_resultado_tv)]
    i2_t0 = duracion_to_entero[float(duracion_resultado_t0)]
    
    # Machacamos el compas con el valor calculado 
    #print('acumulador_aux_tv:',acumulador_aux_tv)
    i3_tv = compas_flamenco_to_entero[int(acumulador_aux_tv)]
    i3_t0 = compas_flamenco_to_entero[int(acumulador_aux_t0)]
    
        
    # Añadir PARAMETRO para parametrizar la temperatura variable
    # Pensar si en lugar de incluir un compas de cortes metemos una lista de compases
    
    print ('acumulador variable',acumulador_aux_tv, acumulador_aux_t0)
    if (int(acumulador_aux_tv) not in compas_corte):
        compas_flamenco_vector.append(round(acumulador_aux_tv, 8) )
        print('compas No corte')
        return (i1_tv, i2_tv, i3_tv, compas_flamenco_vector)
    else:
        compas_flamenco_vector.append(round(acumulador_aux_t0, 8) )    
        print('compas Si corte')
        return (i1_t0, i2_t0, i3_t0, compas_flamenco_vector)   

In [728]:
def cargar_prediccion_salida(prediction_salida):
    compas_flamenco_vector=[]
    acumulator_compas=1
    compas_solea=12
    for i in prediction_salida:
        if (acumulator_compas >= compas_solea + 1):
            acumulator_compas= acumulator_compas - compas_solea
      
        compas_flamenco_vector.append(acumulator_compas)
        acumulator_compas = round(acumulator_compas + i[1], 8)
        # En los casos de tresillos, cinquillos, seisillos, etc ajustamos el compas por los decimales
        if (int(acumulator_compas + 0.01) == round(acumulator_compas,1)):
            acumulator_compas=int(acumulator_compas + 0.01)

    return compas_flamenco_vector

In [729]:
prediccion_entrada=[]
aplicar_ajuste_temperatura_mejorado = True

# Carga inicial
compas_flamenco_vector=cargar_prediccion_salida(prediccion_salida)

# Contador de cambios para estadisticas
cambios_notas=0
cambios_duraciones=0
cambios_compas=0

for note_index in range(max_extra_notas):
    
    # Concatenamos la prediccion en una lista para pasarsela al modelo
    prediccion_entrada = [np.array([notas_entrada_sequence]), np.array([duraciones_entrada_sequence]), np.array([compas_flamenco_entrada_sequence])]

    print('entrada_sequence:', entero_to_nota[notas_entrada_sequence[-1]], 
           '--', entero_to_duracion[duraciones_entrada_sequence[-1]],
           '--', entero_to_compas_flamenco[compas_flamenco_entrada_sequence[-1]])
    
    # Prediccion
    notas_prediction, duraciones_prediction, compas_flamenco_prediction= model.predict(prediccion_entrada, verbose=0)
    
    # Aplicamos algoritmo de temperatura a la prediccion
    #i1 = temperatura_adhoc(notas_prediction[0], notas_temp, prediccion_entrada, notas_prediction, duraciones_prediction, compas_flamenco_prediction, prediccion_salida)
    
    i1_tv, i1_t0 = mecanismo_temperatura(notas_prediction[0], notas_temp)
    i2_tv, i2_t0 = mecanismo_temperatura(duraciones_prediction[0], duracion_temp)
    i3_tv, i3_t0 = mecanismo_temperatura(compas_flamenco_prediction[0], compas_flamenco_temp)
 
    print('Prediccion antes temperatura:',entero_to_nota[i1_tv], '-', entero_to_nota[i1_t0],
          '-->duracion:', entero_to_duracion[i2_tv], '-',entero_to_duracion[i2_t0],
          '-->compas:',entero_to_compas_flamenco[i3_tv], '-',entero_to_compas_flamenco[i3_t0])
    #######################################
    # NUEVO ajustamos con nueva temperatura
    # En este punto prediccion_salida contiene lo generado hasta ahora sin incluir la nueva nota
    if (aplicar_ajuste_temperatura_mejorado== True):
        i1, i2, i3, compas_flamenco_vector =ajuste_temperatura_variable(i1_tv, i1_t0, compas_flamenco_vector, notas_entrada_sequence, compas_corte=[8,9,10,11,12])
    else:
        i1, i2, i3 = i1_tv, i2_tv, i3_tv
    
    print('Prediccion despues temperatura:',entero_to_nota[i1], 
          '-->duracion:', entero_to_duracion[i2], 
          '-->compas:',entero_to_compas_flamenco[i3])
    

    if (i1!=i1_t0):
        cambios_notas=cambios_notas+1
        print('cambio en nota:', entero_to_nota[i1], entero_to_nota[i1_t0] )
    if (i2!=i2_t0):
        cambios_duraciones=cambios_duraciones+1
        print('cambio en duracion:', entero_to_duracion[i2], entero_to_duracion[i2_t0])
    if (i3!=i3_t0):
        cambios_compas=cambios_compas+1
        print('cambio en compas:', entero_to_compas_flamenco[i3], entero_to_compas_flamenco[i3_t0],)

        
    #######################################
    
    # Convertimos de entero a nota
    nota_resultado = entero_to_nota[i1]
    duracion_resultado = entero_to_duracion[i2]
    compas_flamenco_resultado= entero_to_compas_flamenco[i3]
    
    # Añadimos la prediccion a la secuencia de predicciones
    prediccion_salida.append([nota_resultado, duracion_resultado, compas_flamenco_resultado])
    
    # NUEVO: 
    #print('prediccion_salida_append:', prediccion_salida)
    #print('compas_flamenco_vector:', compas_flamenco_vector)
    #print('===============================================')
    
   
    notas_entrada_sequence.append(i1)
    duraciones_entrada_sequence.append(i2)
    compas_flamenco_entrada_sequence.append(i3)
    
    # Si la longitud de la secuencia de entrada debe ser igual que  max_longitud_secuencia 
    # en caso de que sea mayor que max_longitud_secuencia movemos una posicion para que esa salida forme parte de la entrada
    if len(notas_entrada_sequence) > max_longitud_secuencia:
        notas_entrada_sequence = notas_entrada_sequence[1:]
        duraciones_entrada_sequence = duraciones_entrada_sequence[1:]
        compas_flamenco_entrada_sequence = compas_flamenco_entrada_sequence[1:]
        
    print('nota_resultado:', nota_resultado,'duracion_resultado:', duracion_resultado,'compas_flamenco_resultado', compas_flamenco_resultado)
    print('-------------------------------------------------------------------')
    
    
    # Si genera una nota de comienzo PARAMOS la ejecución. Esto pasará en modelo poco entrenados
    if nota_resultado == 'START':
        break


entrada_sequence: F3|0.25 -- 0.25 -- 1
Prediccion antes temperatura: G#3|0.25 - G#3|0.25 -->duracion: 0.25 - 0.25 -->compas: 1 - 1
acumulador variable 1.25 1.25
compas No corte
Prediccion despues temperatura: G#3|0.25 -->duracion: 0.25 -->compas: 1
nota_resultado: G#3|0.25 duracion_resultado: 0.25 compas_flamenco_resultado 1
-------------------------------------------------------------------
entrada_sequence: G#3|0.25 -- 0.25 -- 1
Prediccion antes temperatura: E3|0.25 - E3|0.25 -->duracion: 0.25 - 0.25 -->compas: 1 - 1
acumulador variable 1.5 1.5
compas No corte
Prediccion despues temperatura: E3|0.25 -->duracion: 0.25 -->compas: 1
nota_resultado: E3|0.25 duracion_resultado: 0.25 compas_flamenco_resultado 1
-------------------------------------------------------------------
entrada_sequence: E3|0.25 -- 0.25 -- 1
Prediccion antes temperatura: E4|0.25 - E4|0.25 -->duracion: 0.25 - 0.25 -->compas: 1 - 1
acumulador variable 1.75 1.75
compas No corte
Prediccion despues temperatura: E4|0.25 

Prediccion despues temperatura: F2|0.33333333 -->duracion: 0.33333333 -->compas: 7
cambio en nota: F2|0.33333333 F2|0.25
cambio en duracion: 0.33333333 0.25
nota_resultado: F2|0.33333333 duracion_resultado: 0.33333333 compas_flamenco_resultado 7
-------------------------------------------------------------------
entrada_sequence: F2|0.33333333 -- 0.33333333 -- 7
Prediccion antes temperatura: G#2|0.33333333 - G#2|0.33333333 -->duracion: 0.33333333 - 0.33333333 -->compas: 8 - 8
acumulador variable 7.83333333 7.83333333
compas No corte
Prediccion despues temperatura: G#2|0.33333333 -->duracion: 0.33333333 -->compas: 7
cambio en compas: 7 8
nota_resultado: G#2|0.33333333 duracion_resultado: 0.33333333 compas_flamenco_resultado 7
-------------------------------------------------------------------
entrada_sequence: G#2|0.33333333 -- 0.33333333 -- 7
Prediccion antes temperatura: F2|0.33333333 - F2|0.33333333 -->duracion: 0.33333333 - 0.33333333 -->compas: 8 - 8
acumulador variable 8.16666666 

Prediccion antes temperatura: A3|0.33333333 - E3|0.33333333 -->duracion: 0.33333333 - 0.33333333 -->compas: 2 - 2
acumulador variable 2.83333329 2.83333329
compas No corte
Prediccion despues temperatura: A3|0.33333333 -->duracion: 0.33333333 -->compas: 2
cambio en nota: A3|0.33333333 E3|0.33333333
nota_resultado: A3|0.33333333 duracion_resultado: 0.33333333 compas_flamenco_resultado 2
-------------------------------------------------------------------
entrada_sequence: A3|0.33333333 -- 0.33333333 -- 2
Prediccion antes temperatura: E3|0.33333333 - G3|0.33333333 -->duracion: 0.33333333 - 0.33333333 -->compas: 3 - 3
acumulador variable 3.16666662 3.16666662
compas No corte
Prediccion despues temperatura: E3|0.33333333 -->duracion: 0.33333333 -->compas: 3
cambio en nota: E3|0.33333333 G3|0.33333333
nota_resultado: E3|0.33333333 duracion_resultado: 0.33333333 compas_flamenco_resultado 3
-------------------------------------------------------------------
entrada_sequence: E3|0.33333333 -- 0.

Prediccion antes temperatura: A3|0.33333333 - A3|0.33333333 -->duracion: 0.33333333 - 0.33333333 -->compas: 8 - 8
acumulador variable 8.66666666 8.66666666
compas Si corte
Prediccion despues temperatura: A3|0.33333333 -->duracion: 0.33333333 -->compas: 8
nota_resultado: A3|0.33333333 duracion_resultado: 0.33333333 compas_flamenco_resultado 8
-------------------------------------------------------------------
entrada_sequence: A3|0.33333333 -- 0.33333333 -- 8
Prediccion antes temperatura: C4|0.33333333 - C4|0.33333333 -->duracion: 0.33333333 - 0.33333333 -->compas: 9 - 9
acumulador variable 9 9
compas Si corte
Prediccion despues temperatura: C4|0.33333333 -->duracion: 0.33333333 -->compas: 9
nota_resultado: C4|0.33333333 duracion_resultado: 0.33333333 compas_flamenco_resultado 9
-------------------------------------------------------------------
entrada_sequence: C4|0.33333333 -- 0.33333333 -- 9
Prediccion antes temperatura: B3|0.33333333 - B3|0.33333333 -->duracion: 0.33333333 - 0.3333

Prediccion antes temperatura: G4|0.25 - A3|0.25 -->duracion: 0.25 - 0.25 -->compas: 4 - 4
acumulador variable 4.33333333 4.33333333
compas No corte
Prediccion despues temperatura: G4|0.25 -->duracion: 0.25 -->compas: 4
cambio en nota: G4|0.25 A3|0.25
nota_resultado: G4|0.25 duracion_resultado: 0.25 compas_flamenco_resultado 4
-------------------------------------------------------------------
entrada_sequence: G4|0.25 -- 0.25 -- 4
Prediccion antes temperatura: F4|0.25 - F4|0.25 -->duracion: 0.25 - 0.25 -->compas: 4 - 4
acumulador variable 4.58333333 4.58333333
compas No corte
Prediccion despues temperatura: F4|0.25 -->duracion: 0.25 -->compas: 4
nota_resultado: F4|0.25 duracion_resultado: 0.25 compas_flamenco_resultado 4
-------------------------------------------------------------------
entrada_sequence: F4|0.25 -- 0.25 -- 4
Prediccion antes temperatura: D4|0.25 - D4|0.25 -->duracion: 0.25 - 0.25 -->compas: 4 - 4
acumulador variable 4.83333333 4.83333333
compas No corte
Prediccion des

...

Prediccion antes temperatura: B3|0.33333333 - B3|0.33333333 -->duracion: 0.33333333 - 0.33333333 -->compas: 12 - 12
acumulador variable 12.83333318 12.83333318
compas Si corte
Prediccion despues temperatura: B3|0.33333333 -->duracion: 0.33333333 -->compas: 12
nota_resultado: B3|0.33333333 duracion_resultado: 0.33333333 compas_flamenco_resultado 12
-------------------------------------------------------------------
entrada_sequence: B3|0.33333333 -- 0.33333333 -- 12
Prediccion antes temperatura: A3|0.33333333 - A3|0.33333333 -->duracion: 0.33333333 - 0.33333333 -->compas: 1 - 1
acumulador variable 1.1666665100000007 1.1666665100000007
compas No corte
Prediccion despues temperatura: A3|0.33333333 -->duracion: 0.33333333 -->compas: 1
nota_resultado: A3|0.33333333 duracion_resultado: 0.33333333 compas_flamenco_resultado 1
-------------------------------------------------------------------
entrada_sequence: A3|0.33333333 -- 0.33333333 -- 1
Prediccion antes temperatura: G3|0.33333333 - G3|0.

Prediccion antes temperatura: G#3|0.25 - C4|0.25 -->duracion: 0.25 - 0.25 -->compas: 7 - 7
acumulador variable 7.66666647 7.66666647
compas No corte
Prediccion despues temperatura: G#3|0.25 -->duracion: 0.25 -->compas: 7
cambio en nota: G#3|0.25 C4|0.25
nota_resultado: G#3|0.25 duracion_resultado: 0.25 compas_flamenco_resultado 7
-------------------------------------------------------------------
entrada_sequence: G#3|0.25 -- 0.25 -- 7
Prediccion antes temperatura: F3|0.25 - F3|0.25 -->duracion: 0.25 - 0.25 -->compas: 8 - 8
acumulador variable 7.91666647 7.91666647
compas No corte
Prediccion despues temperatura: F3|0.25 -->duracion: 0.25 -->compas: 7
cambio en compas: 7 8
nota_resultado: F3|0.25 duracion_resultado: 0.25 compas_flamenco_resultado 7
-------------------------------------------------------------------
entrada_sequence: F3|0.25 -- 0.25 -- 7
Prediccion antes temperatura: C3|0.33333333 - C3|0.33333333 -->duracion: 0.33333333 - 0.33333333 -->compas: 8 - 8
acumulador variable 8

Prediccion antes temperatura: B-3|0.33333333 - A3|0.25 -->duracion: 0.25 - 0.25 -->compas: 1 - 1
acumulador variable 1.58333309 1.58333309
compas No corte
Prediccion despues temperatura: B-3|0.33333333 -->duracion: 0.33333333 -->compas: 1
cambio en nota: B-3|0.33333333 A3|0.25
cambio en duracion: 0.33333333 0.25
nota_resultado: B-3|0.33333333 duracion_resultado: 0.33333333 compas_flamenco_resultado 1
-------------------------------------------------------------------
entrada_sequence: B-3|0.33333333 -- 0.33333333 -- 1
Prediccion antes temperatura: B-3|0.33333333 - F2|0.33333333 -->duracion: 0.33333333 - 0.33333333 -->compas: 2 - 2
acumulador variable 1.91666642 1.91666642
compas No corte
Prediccion despues temperatura: B-3|0.33333333 -->duracion: 0.33333333 -->compas: 1
cambio en nota: B-3|0.33333333 F2|0.33333333
cambio en compas: 1 2
nota_resultado: B-3|0.33333333 duracion_resultado: 0.33333333 compas_flamenco_resultado 1
--------------------------------------------------------------

Prediccion antes temperatura: D4|0.25 - D4|0.25 -->duracion: 0.25 - 0.25 -->compas: 8 - 8
acumulador variable 8.5 8.5
compas Si corte
Prediccion despues temperatura: D4|0.25 -->duracion: 0.25 -->compas: 8
nota_resultado: D4|0.25 duracion_resultado: 0.25 compas_flamenco_resultado 8
-------------------------------------------------------------------
entrada_sequence: D4|0.25 -- 0.25 -- 8
Prediccion antes temperatura: B-4|0.25 - A3|0.25 -->duracion: 0.25 - 0.25 -->compas: 8 - 8
acumulador variable 8.75 8.75
compas Si corte
Prediccion despues temperatura: A3|0.25 -->duracion: 0.25 -->compas: 8
nota_resultado: A3|0.25 duracion_resultado: 0.25 compas_flamenco_resultado 8
-------------------------------------------------------------------
entrada_sequence: A3|0.25 -- 0.25 -- 8
Prediccion antes temperatura: E4|0.5 - E-4|0.25 -->duracion: 0.25 - 0.25 -->compas: 9 - 9
acumulador variable 9 9
compas Si corte
Prediccion despues temperatura: E-4|0.25 -->duracion: 0.25 -->compas: 9
nota_resultado: E

Prediccion antes temperatura: F3|0.25 - B-3|0.33333333 -->duracion: 0.33333333 - 0.33333333 -->compas: 4 - 4
acumulador variable 3.91666665 3.91666665
compas No corte
Prediccion despues temperatura: F3|0.25 -->duracion: 0.25 -->compas: 3
cambio en nota: F3|0.25 B-3|0.33333333
cambio en duracion: 0.25 0.33333333
cambio en compas: 3 4
nota_resultado: F3|0.25 duracion_resultado: 0.25 compas_flamenco_resultado 3
-------------------------------------------------------------------
entrada_sequence: F3|0.25 -- 0.25 -- 3
Prediccion antes temperatura: G#3|0.33333333 - E2|0.5 -->duracion: 0.5 - 0.5 -->compas: 4 - 4
acumulador variable 4.16666665 4.16666665
compas No corte
Prediccion despues temperatura: G#3|0.33333333 -->duracion: 0.33333333 -->compas: 4
cambio en nota: G#3|0.33333333 E2|0.5
cambio en duracion: 0.33333333 0.5
nota_resultado: G#3|0.33333333 duracion_resultado: 0.33333333 compas_flamenco_resultado 4
-------------------------------------------------------------------


In [730]:
print('cambios_notas:',cambios_notas)
print('cambios_duraciones:',cambios_duraciones)
print('cambios_compas:',cambios_compas)

cambios_notas: 109
cambios_duraciones: 56
cambios_compas: 40


In [731]:
from music21 import *
import time
directorio_salida = os.path.join(directorio_ejecucion, 'output')

midi_stream = stream.Stream()

# Creación de notas y acordes en base a las notas generadas por el modelo
for patron in prediccion_salida:
    
    #nota_patron, duracion_patron, compas_flamenco_patron = patron
    nota_patron_aux, duracion_patron, compas_flamenco_patron = patron
    print("nota_patron_aux:", nota_patron_aux)
    nota_patron, x = nota_patron_aux.split("|")
    
    # patron es un chord (acorde) porque lleva varias notas separadas por un . 'G3.F#3.A3'
    if ('.' in nota_patron):
        notas_acorde = nota_patron.split('.')
        acorde_notas = []
        for nota_actual in notas_acorde:
            nueva_nota = note.Note(nota_actual)
            nueva_nota.duration = duration.Duration(duracion_patron)
            nueva_nota.storedInstrument = instrument.Violoncello()
            acorde_notas.append(nueva_nota)
        nuevo_acorde = chord.Chord(acorde_notas)
        midi_stream.append(nuevo_acorde)
    elif nota_patron == 'rest':
    # patron es un rest (Un silencia)
        nueva_nota = note.Rest()
        nueva_nota.duration = duration.Duration(duracion_patron)
        nueva_nota.storedInstrument = instrument.Violoncello()
        midi_stream.append(nueva_nota)
    elif nota_patron != 'START':
    # patron es una note (en cualquier otro caso en una nota sencilla)
        nueva_nota = note.Note(nota_patron)
        nueva_nota.duration = duration.Duration(duracion_patron)
        nueva_nota.storedInstrument = instrument.Guitar()
        midi_stream.append(nueva_nota)


# Guardamos las notas generadas en un fichero en formato MIDI
print('midi_stream', midi_stream[0])
midi_stream= midi_stream.chordify()
print('midi_stream', midi_stream[0])
timestr = time.strftime("%Y%m%d-%H%M%S")
midi_stream.write('midi', fp=os.path.join(directorio_salida, 'output-' + timestr + '.mid'))

nota_patron_aux: START|0.0
nota_patron_aux: START|0.0
nota_patron_aux: START|0.0
nota_patron_aux: START|0.0
nota_patron_aux: START|0.0
nota_patron_aux: START|0.0
nota_patron_aux: START|0.0
nota_patron_aux: START|0.0
nota_patron_aux: START|0.0
nota_patron_aux: START|0.0
nota_patron_aux: START|0.0
nota_patron_aux: START|0.0
nota_patron_aux: START|0.0
nota_patron_aux: START|0.0
nota_patron_aux: START|0.0
nota_patron_aux: F3|0.25
nota_patron_aux: G#3|0.25
nota_patron_aux: E3|0.25
nota_patron_aux: E4|0.25
nota_patron_aux: F4|0.33333333
nota_patron_aux: E4|0.33333333
nota_patron_aux: D3|0.5
nota_patron_aux: D4|0.5
nota_patron_aux: D3|0.25
nota_patron_aux: C3|0.25
nota_patron_aux: E3|0.25
nota_patron_aux: F3|0.25
nota_patron_aux: B3|0.25
nota_patron_aux: B2|0.25
nota_patron_aux: F3|0.25
nota_patron_aux: G3|0.25
nota_patron_aux: B3|0.25
nota_patron_aux: A3|0.25
nota_patron_aux: G3|0.25
nota_patron_aux: F#4|0.33333333
nota_patron_aux: B2|0.25
nota_patron_aux: C3|0.25
nota_patron_aux: D3|0.25
no

nota_patron_aux: F3|0.33333333
nota_patron_aux: G3|0.33333333
nota_patron_aux: F3|0.33333333
nota_patron_aux: E3|0.33333333
nota_patron_aux: D3|0.33333333
nota_patron_aux: B2|0.33333333
nota_patron_aux: C3|0.33333333
nota_patron_aux: D3|0.33333333
nota_patron_aux: C3|0.33333333
nota_patron_aux: B2|0.33333333
nota_patron_aux: A2|0.33333333
nota_patron_aux: F2|0.33333333
nota_patron_aux: G2|0.33333333
nota_patron_aux: A2|0.33333333
nota_patron_aux: G#2|0.33333333
nota_patron_aux: G2|0.33333333
nota_patron_aux: F2|0.33333333
nota_patron_aux: E2|0.5
nota_patron_aux: E4|0.5
nota_patron_aux: G#3|0.5
nota_patron_aux: B3|0.5
nota_patron_aux: E-3|0.25
nota_patron_aux: F3|0.25
nota_patron_aux: G#3|0.25
nota_patron_aux: E4|0.25
nota_patron_aux: C4|0.25
nota_patron_aux: E4|0.25
nota_patron_aux: C4|0.25
nota_patron_aux: B3|0.5
nota_patron_aux: G#3|0.25
nota_patron_aux: D4|0.5
nota_patron_aux: F4|0.25
nota_patron_aux: E4|0.25
nota_patron_aux: D4|0.25
nota_patron_aux: C4|0.25
nota_patron_aux: E4|0.25

'./ejecucion/composicion/0001_guitarra\\output\\output-20210620-103317.mid'